# <span style="color:red;">NOTE</span>: Use the ALL_STEPS notebook to run all subjects. 

* `dti_diffeomorphic_population` on one subject will give `mean_df not found error`. Use `dti_diffeomorphic_reg` for one subject.


# DTI Image Registration Using Diffusion Tensor Imaging ToolKit (DTI-TK)
## This notebook is my way of trying out DTI-TK image registration steps explained [here](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.HomePage)
## I created this notebook after [this](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.FirstRegistration) example tutorial. This notebook will perform the suggested steps by DTI-TK for correct image registration.

> should proceed by
>    * Reading the tutorials on data format, visualization and checking diffusivity unit
>    * Going through the detailed preprocessing tutorial
>    * Studying the detailed registration tutorial.

# BASIC PROCESSING AND VISUALIZATION 

## [Interoperability in DTI-TK with FSL](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Interoperability)
* Interoperability with FSL is straighforward using the `fsl_to_dtitk` script that converts a set of V[1-3] and L[1-3] dtifit files into the NIfTI tensor format that DTI-TK uses.

* To convert back to the FSL format, use TVEigenSystem, e.g.

        TVEigenSystem -in tensor.nii.gz -type FSL 

In [1]:
import os
from pathlib import Path
from pgimri.config import PROCESSED_DTI_FILENAME

In [2]:
# This is similar to running following commands in terminal
# export DTITK_ROOT=../dtitk
# export PATH=${PATH}:${DTITK_ROOT}/bin:${DTITK_ROOT}/utilities:${DTITK_ROOT}/scripts
dtitk_maindir = "../dtitk"
os.environ["DTITK_ROOT"] = dtitk_maindir
os.environ["PATH"] += f":{dtitk_maindir}/bin:{dtitk_maindir}/utilities:{dtitk_maindir}/scripts"
!which VolumeInfo

../dtitk/bin/VolumeInfo


> * In practice, we recommend the users to use fsl_to_dtitk, a script that takes a FSL-generated DTI eigensystem volumes (non-brain tissue already removed) and converts them into fully DTI-TK compatible DTI volumes, i.e., both in the correct format and correctly preprocessed.
> * The output from this program will be tensor_L{1,2,3}.nii.gz, the eigenvalues, and tensor_V{1,2,3}.nii.gz, the eigenvectors, following precisely the FSL naming convention.


> * **WARNING**: FSL also has an option that allows for a DTI volume to be saved on disk in matrix form similar to but NOT the same as the NIfTI tensor format. It stores the six independent matrix components using the "upper triangular" order rather than the "lower triangular" order specified in NIfTI standard! (See Section about CAMINO above for definition of upper and lower triangular) So DO NOT use this option if you want to use its output with DTI-TK!

In [3]:
!ls

 DTI_ITC.ipynb				      dti_V2.nii.gz
 dti_L1.nii.gz				      dti_V3.nii.gz
 dti_L2.nii.gz				      reference_axial_tensor.png
 dti_L3.nii.gz				      reference_coronal_tensor.png
'DTI-TK image registration_ALL_STEPS.ipynb'   subjects_list_aff_diffeo.txt
'DTI-TK image registration example.ipynb'     subjects_list_aff.txt
'DTI-TK image registration.ipynb'	      subjects_list.txt
 dti_V1.nii.gz


In [4]:
# Convert FSL format to DTI-TK format for further processing
# base_filepath = f"../../data/DTI/processed_data/3_processed/S63080_HARSH_ITS/{PROCESSED_DTI_FILENAME}"
base_filepath = PROCESSED_DTI_FILENAME
!fsl_to_dtitk {base_filepath} # The output basename will be `<PROCESSED_DTI_FILENAME>_dtitk...`

Reading dti_L1.nii.gz ... Done in 0.007444s
Reading dti_V1.nii.gz ... Done in 0.021673s
Reading dti_L2.nii.gz ... Done in 0.007762s
Reading dti_V2.nii.gz ... Done in 0.021025s
Reading dti_L3.nii.gz ... Done in 0.006831s
Reading dti_V3.nii.gz ... Done in 0.02126s
Writing dti_dtitk.nii.gz ... Done in 0.182574s
Reading dti_dtitk.nii.gz ... Done in 0.039572s
Voxelwise scaling dti_dtitk.nii.gz by 1000 ... Done in 0.003551s
Writing dti_dtitk.nii.gz ... Done in 0.194446s
Reading dti_dtitk.nii.gz ... Done in 0.043771s
Computing the tensor Euclidean norm map ... Done in 0.00448s
Writing dti_dtitk_norm.nii.gz ... Done in 0.035147s
Reading dti_dtitk.nii.gz ... Done in 0.041721s
Reading dti_dtitk_norm_non_outliers.nii.gz ... Done in 0.002562s
Masking dti_dtitk.nii.gz by dti_dtitk_norm_non_outliers.nii.gz ... Done in 0.000822s
Writing dti_dtitk.nii.gz ... Done in 0.193877s
Reading dti_dtitk.nii.gz ... Done in 0.042983s
Converting dti_dtitk.nii.gz to be symmetric positive definite ... Done in 0.0776

In [5]:
!ls

 dti_dtitk.nii.gz			     'DTI-TK image registration.ipynb'
 dti_dtitk_nonSPD.nii.gz		      dti_V1.nii.gz
 dti_dtitk_norm.nii.gz			      dti_V2.nii.gz
 dti_dtitk_norm_non_outliers.nii.gz	      dti_V3.nii.gz
 DTI_ITC.ipynb				      reference_axial_tensor.png
 dti_L1.nii.gz				      reference_coronal_tensor.png
 dti_L2.nii.gz				      subjects_list_aff_diffeo.txt
 dti_L3.nii.gz				      subjects_list_aff.txt
'DTI-TK image registration_ALL_STEPS.ipynb'   subjects_list.txt
'DTI-TK image registration example.ipynb'


## [VISUALIZING DTI IMAGES WITH DTI-TK](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.VisualizationTool)

### Below are reference views of how FA should look like:

Axial View            |  Cornonal View
:-------------------------:|:-------------------------:
![reference_axial_tensor.png](reference_axial_tensor.png)  |  ![reference_coronal_tensor.png](reference_coronal_tensor.png)

### Checking if tensors are correctly reconstructed
An important utility of the tensor glyph rendering described above is for checking the correctness of your tensor reconstruction. A common mistake during tensor reconstruction (the process of constructing tensor volumes from diffusion-weighted images) is using a gradient table that is not corrected for the difference between the coordinate frame in which the gradient table is defined and the one in which the image is defined. This type of erroneous reconstruction can not be detected with the RGB map (explained above) and need to be ruled out with the tensor glyph rendering.

The two steps are:

1. Check if the tensors are oriented correctly in the genu and the splenium of the corpus callosum in the axial view (shown above). The tensors' major axes should be clearly aligned along the boundary of the genu and the splenium.
2. Check if the tensors are oriented correctly in the internal capsule and the midbody of the corpus callosum in the coronal view (also shown above). Again, the tensors should be properly aligned with the boundary of these structures.

In [6]:
# Check the new volumes
dtitk_path = f"{base_filepath}_dtitk.nii.gz"

# Display basic volume info
!VolumeInfo {dtitk_path}

# Viewing the tensors using 3D ellipsoid glyphs
!TVglyphView -in {dtitk_path}

# Other ways to visualize
# !TVglyphView -in {dtitk_path} -scale 2 -view axial
# !TVglyphView -in {dtitk_path} -scale 2 -view coronal

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: LPI
Volume Info of dti_dtitk.nii.gz
size: 112x112x48, voxel size: 1.60714x1.60714x2.5, origin: [0, 0, 0]
Reading dti_dtitk.nii.gz ... Done in 0.056838s
press the key 'H' to print a list of the available control keystrokes
total number of colors = 262144
processing the selected axial slice ... done rendering 


# SPATIAL NORMALIZATION AND ATLAS CONSTRUCTION 

## [PREPROCESSING OF VOLUMES BEFORE REGISTRATION](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.BeforeReg)

### [**IMP**] Make Sure DTI Volumes are Using the Correct Diffusivity Unit. [More details here](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Diffusivity)
* **The unit used by DTI-TK is $10^{-3}mm^2.s^{-1}$**. With this unit, the mean diffusivity for CSF is around 3. You want to make sure your DTI volumes are stored in the correct unit.

* for diffusion-weighting, b-value of 800 s.mm-2 is used during the acquisition, and you input the number 800 as the b-value to your tensor reconstruction program, then the output DTI volume will have the unit of mm2.s-1.

* if your DTI volumes use the unit of mm2.s-1, then the multiplication factor should be 1,000. If your unit is m2.s-1, then the factor should be $10^9$.

In [8]:
# Unit of Diffusivity correction
dtitk_scaled_path = dtitk_path.replace("_dtitk.nii.gz", "_dtitk_scaled.nii.gz")
!TVtool -in {dtitk_path} -scale 1000 -out {dtitk_scaled_path}

Reading dti_dtitk.nii.gz ... Done in 0.040787s
Voxelwise scaling dti_dtitk.nii.gz by 1000 ... Done in 0.003373s
Writing dti_dtitk_scaled.nii.gz ... Done in 0.194424s


In [10]:
# Check scaled file

#! [WARNING]: Do not run without scaling down. System will hang because of too heavy rendering.
# !TVglyphView -in {dtitk_scaled_path} -scale 0.001

### Remove Non-Brain Tissues (Already completed this step in `process-subject` function)
To do this, our recommended protocol is to use the excellent Brain Extraction Tool, aka BET, available as part of the FSL suite. Once you have computed the brain tissue mask and are satisfied with its quality, you can apply it to the corresponding DTI volume with the following command:

        TVtool -in tensor.nii.gz -out mtensor.nii.gz -mask b0_mask.nii.gz

Comment:
* A good quality brain mask should not remove any brain tissues and keep non-brain tissue at a minimum.
* DTI-TK expects b0_mask.nii.gz to be 0 for the background and 1 for brain tissues.

In [8]:
#! TRY ADDING THIS STEP TO SEE IF ANYTHING IMPROVES

### Make Sure DTI Volumes are SPD
[**NOTE**] Adding this step before norm because norm volume does not process SPD. So, norm will be applied after SPD.
Diffusion tensors, by definition, are symmetric and positive-definite matrices, or SPD. However, in practice, due to noises in the DWI images, the diffusion tensors estimated within a DTI volume are not always SPD. The SPD condition is very important however to ensure DTI volumes behave properly after various image processing steps. DTI-TK provides a simple tool that identifies the voxels that are not SPD and enforces this condition on such voxels. The tool TVtool can be used as follows:

        TVtool -in tensor.nii.gz -spd -out spd.nii.gz

In [23]:
dtitk_spd_path = dtitk_scaled_path.replace("dtitk_scaled.nii.gz", "dtitk_scaled_spd.nii.gz")

# Just make sure from the output that tensor count in non-spd file is zero.
!TVtool -in {dtitk_scaled_path} -spd -out {dtitk_spd_path}

Reading dti_dtitk_scaled.nii.gz ... Done in 0.041277s
Converting dti_dtitk_scaled.nii.gz to be symmetric positive definite ... Done in 0.07504s
Writing dti_dtitk_scaled_nonSPD.nii.gz ... Done in 0.01258s
non-spd tensors count = 0
Writing dti_dtitk_scaled_spd.nii.gz ... Done in 0.183265s


### Make Sure DTI Volumes Contain No Significant Outliers
Outliers often don't have any effect on image registration. However, when the outliers are substantially outside the normal range, they will. A good indicator that the outliers are worthy of concerns is when their existence distorts the mean significantly.

        TVtool -in tensor.nii.gz -norm
which outputs the tensor norm image as tensor_norm.nii.gz, and

        SVtool -in tensor_norm.nii.gz -stats

In [16]:
# # Normalize
# !TVtool -in {dtitk_spd_path} -norm

# dtitk_norm_path = dtitk_spd_path.replace("dtitk_scaled_spd.nii.gz", "dtitk_scaled_spd_norm.nii.gz")

# print("Saved @", dtitk_norm_path)


### Check Whether DTI Volumes Share one Common Voxel Space
It's recommended to set all the origin to [0, 0, 0] and this can be done with "TVAdjustVoxelspace".

        TVAdjustVoxelspace -in tensor.nii.gz -origin 0 0 0

In [17]:
# Display basic volume info (it is alerady there, but do include these steps anyway in the registration pipeline)
!VolumeInfo {dtitk_norm_path}

NIFTI Intent Code: NIFTI_INTENT_NONE
NIFTI Orientation Code: LPI
Volume Info of dti_dtitk_scaled_spd_norm.nii.gz
size: 112x112x48, voxel size: 1.60714x1.60714x2.5, origin: [0, 0, 0]


In [13]:
# Visualize
# !TVglyphView -in {dtitk_spd_path} -scale 0.001

Reading dti_dtitk_scaled_spd_norm.nii.gz ... Warning: Volume uses unexpected NIfTI intent code
Done in 0.007372s


It is important to resample the volumes that do not share the same voxel spacing as the rest of the volumes. This is usually never an issue unless you are trying to combine data acquired with different protocols. The tool for resampling tensor volumes is TVResample.
It's ideal but not necessary that all the volumes have the same voxel dimensions. This situation may arise when the same protocol is used if the operator decides to optimze the number of slices to acquire. Not having the same voxel dimensions (z dimension in most cases) may reduce the initial bootstrapped template estimate. Visual inspection can be used to verify if this is a serious issue or not.

## SHOULD I DO RESAMPLING? (check back after trying registration for any significant difference.)

## [REGISTRATION AND SPATIAL NORMALIZATION OF DTI VOLUMES](http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Registration)

The spatial normalization pipeline includes the following five steps:

1. Preprocessing of the input DTI volumes
2. Bootstrapping the initial DTI template from the input DTI volumes.
3. Rigid alignment with template refinement
4. Affine alignment with template refinement
5. Deformable alignment with template refinement

Where step 1 is already performed above.

#### Step 2. Bootstrapping the initial DTI template from the input DTI volumes

In [34]:
template_path = "../../nihpd_templates/nihpd_asym_00-02_t1w.nii"
template_mask_path = "../../nihpd_templates/nihpd_asym_00-02_mask.nii"
# template_path = "ixi_aging_template.nii.gz"
# template_mask_path = "ixi_aging_template_brain_mask.nii.gz"

# Create a subject list file
with open("subjects_list.txt", 'w') as wf:
    wf.write(f"{Path(dtitk_spd_path).resolve().absolute()}\n")

In [35]:
!VolumeInfo {template_path}

NIFTI Intent Code: NIFTI_INTENT_NONE
NIFTI Orientation Code: RPI
Volume Info of ../../nihpd_templates/nihpd_asym_00-02_t1w.nii
size: 195x233x159, voxel size: 1x1x1, origin: [-0, -0, -0]


In [38]:
# !asvDSM -template {template_path} -subject mean_initial.nii.gz -sep 0.5 0.5 0.5 -ftol 0.0001

Affine Registration of mean_initial.nii.gz to ../../nihpd_templates/nihpd_asym_00-02_t1w.nii
Reading ../../nihpd_templates/nihpd_asym_00-02_t1w.nii ... Done in 0.0054s
sampling step size = [1, 1, 1]
Reading mean_initial.nii.gz ... Warning: Volume uses unexpected NIfTI intent code
Done in 0.461312s
Using Normalized Mutual Information metric
Initial difference = -0.0761274
initial estimate provided by center of mass alignment
dsm optimizer ftol = 0.0001
Direction Set Minimization ...
Iteration 0
 -1.01  5.645 -0.499      0      0      0      0      0      0      0      0      0 | -0.0762226
 -1.01  1.409 -0.499      0      0      0      0      0      0      0      0      0 | -0.0767918
 -1.01  1.409  -1.55      0      0      0      0      0      0      0      0      0 | -0.0769662
 -1.01  1.409  -1.55 0.03231      0      0      0      0      0      0      0      0 | -0.0769677
 -1.01  1.409  -1.55 0.03231 0.004255      0      0      0      0      0      0      0 | -0.0769684
 -1.01  1.40

Iteration 6
-0.3924  10.81  11.95  13.47 -1.218 -0.202  0.179 0.2362 -7.104 -0.09958 0.4314  1.673 | -0.0789281
-0.3924  10.62  11.95  13.47 -1.218 -0.202  0.179 0.2362 -7.104 -0.09958 0.4314  1.673 | -0.0789291
-0.3924  10.62  11.95  13.47 -1.218 -0.202  0.179 0.2362 -7.104 -0.09958 0.4314  1.843 | -0.078945
-0.3531  11.01  12.43  13.77  -1.22 -0.1765 0.1954  0.203 -7.237 -0.0996 0.4324  1.843 | -0.078952
-0.3531  11.01  12.43  13.77 -1.193 -0.1765 0.1954  0.203 -7.237 -0.0996 0.4324  1.843 | -0.0789521
-0.3531  11.01  12.43  13.77 -1.193 -0.3037 0.1954  0.203 -7.237 -0.0996 0.4324  1.843 | -0.0789559
-0.3531  11.01  12.43  13.77 -1.193 -0.3037 0.1783  0.203 -7.237 -0.0996 0.4324  1.843 | -0.0789568
-0.3531  11.01  12.43  13.77 -1.193 -0.3037 0.1783 0.1906 -7.237 -0.0996 0.4324  1.843 | -0.0789572
-0.3531  11.01  12.43  13.77 -1.193 -0.3037 0.1783 0.1906 -7.237 -0.0996 0.4324  1.843 | -0.0789573
-0.3531  11.01  12.43  13.77 -1.193 -0.3037 0.1783 0.1906 -7.237 -0.0996 0.4324  1.843 | -

In [26]:
# Remove non-brain part from template
!TVtool -in {template_path} -out template_0to2.nii.gz -mask {template_mask_path}

Reading ../../nihpd_templates/nihpd_asym_00-02_t1w.nii.gz ... Warning: Volume uses unexpected NIfTI intent code
Done in 0.101272s


In [19]:
#! [WARNING] NORM files does not work. Use others
!dti_template_bootstrap {template_path} subjects_list.txt

dti_rigid_sn iteration 1
registering /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk_scaled_spd.nii.gz ...
Rigid Registration of /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk_scaled_spd.nii.gz to ../../nihpd_templates/nihpd_asym_00-02_t1w.nii
Reading ../../nihpd_templates/nihpd_asym_00-02_t1w.nii ... Done in 0.009476s
sampling step size = [4, 4, 4]
Gaussian smoothing: sigma = [1.64, 1.64, 1.64] ... Done in 0.496s
Reading /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk_scaled_spd.nii.gz ... Warning: Volume uses unexpected NIfTI intent code
Done in 0.0368s
Gaussian smoothing: sigma = [0.968, 0.968, 0.53] ... Done in 0.024s
Using Normalized Mutual Information metric
Initial difference = -0.0289
initial estimate provided by center of mass alignment
dsm optimizer ftol = 0.01
Direction Set Minimization ...
Iteration 0
-8.262    -70 -18.54      0      0      0 | -0.0528492
-8.262 -69.62 -18.54      0      0      0 | -0.0528511
-8.262 -69.62 -50.38      0 

-7.127 -61.82 -21.02  9.928 0.7785 -8.947 -7.009 -5.025 -14.94  1.263  0.807  2.035 | -0.0780386
-7.127 -61.82 -21.02  9.928 0.7785 -8.947 -7.009 -5.025 -16.76  1.263  0.807  2.035 | -0.0788361
-7.127 -61.82 -21.02  9.928 0.7785 -8.947 -7.009 -5.025 -16.76  1.224  0.807  2.035 | -0.0788428
-7.127 -61.82 -21.02  9.928 0.7785 -8.947 -7.009 -5.025 -16.76  1.224 0.7991  2.035 | -0.0788468
 -7.25 -61.67 -18.16  10.74  1.176 -9.021 -7.014 -5.024 -17.67   1.25 0.8622  1.945 | -0.0792658
Iteration 4
-7.387 -61.67 -18.16  10.74  1.176 -9.021 -7.014 -5.024 -17.67   1.25 0.8622  1.945 | -0.079274
-7.387 -60.24 -18.16  10.74  1.176 -9.021 -7.014 -5.024 -17.67   1.25 0.8622  1.945 | -0.0793723
-7.387 -60.24 -18.16  10.74  1.176 -9.021 -7.014 -5.024 -17.67   1.25 0.8622    2.3 | -0.0794914
-7.387 -60.24 -18.16  11.23  1.176 -9.021 -7.014 -5.024 -17.67   1.25 0.8622    2.3 | -0.0795563
-7.387 -60.24 -18.16  11.23 -0.6694 -9.021 -7.014 -5.024 -17.67   1.25 0.8622    2.3 | -0.0796762
-7.387 -60.24 -18.

In [20]:
!ls

 dti_dtitk.nii.gz
 dti_dtitk_nonSPD.nii.gz
 dti_dtitk_norm.nii.gz
 dti_dtitk_norm_non_outliers.nii.gz
 dti_dtitk_scaled.nii.gz
 dti_dtitk_scaled_nonSPD.nii.gz
 dti_dtitk_scaled_spd.aff
 dti_dtitk_scaled_spd_aff.nii.gz
 dti_dtitk_scaled_spd.nii.gz
 dti_dtitk_scaled_spd_norm.nii.gz
 DTI_ITC.ipynb
 dti_L1.nii.gz
 dti_L2.nii.gz
 dti_L3.nii.gz
'DTI-TK image registration_ALL_STEPS.ipynb'
'DTI-TK image registration example.ipynb'
'DTI-TK image registration.ipynb'
 dti_V1.nii.gz
 dti_V2.nii.gz
 dti_V3.nii.gz
 mean_initial.nii.gz
 reference_axial_tensor.png
 reference_coronal_tensor.png
 subjects_list_aff_diffeo.txt
 subjects_list_aff.txt
 subjects_list.txt


In [21]:
# output affine filepath
dtitk_aff_path = dtitk_path.replace("_dtitk.nii.gz", "_dtitk_scaled_spd_aff.nii.gz")

!TVglyphView -in {dtitk_aff_path} -scale 0.001 -view cornonal

Reading dti_dtitk_scaled_spd_aff.nii.gz ... Done in 0.469017s
press the key 'H' to print a list of the available control keystrokes
total number of colors = 262144
processing the selected axial slice ... done rendering 


#### 3. Rigid alignment with template refinement

    dti_rigid_reg template.nii.gz tensor.nii.gz EDS 4 4 4 0.01 1


In [24]:
# !dti_rigid_reg {template_path} {dtitk_path} EDS 4 4 4 0.01

#### 4. Affine alignment with template refinement

In [27]:
!dti_affine_population mean_initial.nii.gz subjects_list.txt MNI 3 # MNI is more robust than EDS

command:  mean_initial.nii.gz subjects_list.txt EDS 3
Sun 15 Aug 2021 03:24:06 PM IST
dti_affine_population iteration 1
dti_affine_sn iteration 1
registering /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.nii.gz ...
Affine Registration of /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.nii.gz to mean_affine0.nii.gz
Similarity Measure : EDS
Reorient Option : FS
Reading mean_affine0.nii.gz ... Done in 0.069863s
sampling step size = [2, 2, 2]
Gaussian smoothing: sigma = [0.873, 0.873, 0.624] ... Done in 0.221s
Reading /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.nii.gz ... Done in 0.0344s
Gaussian smoothing: sigma = [0.968, 0.968, 0.53] ... Done in 0.127s
Initial Difference = 6.45e+06
initial estimate provided by /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.aff
-19.3	-6.76	-14.9	0.196	-0.0252	-0.182	0.896	0.925	0.956	0.0108	0.00741	-0.0521	
cgm optimizer ftol = 0.01
both iteration 0 = 1.28e+05
both iteration 1 = 6.6e+04
both iteration

Reading /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.nii.gz ... Done in 0.039967s
trans (inverse) applied = [ 3.057, -37.811,  9.117]
[ 0.994, -0.179, -0.025;  0.180,  0.973,  0.197; -0.010, -0.202,  0.995]
reading output volume specification from mean_affine2.nii.gz
backward resampling ...time consumed = 0.210
Writing /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk_aff.nii.gz ... Done in 0.236s
done

done
Input affine transformation = [ 3.057, -37.811,  9.117]
[ 0.994, -0.179, -0.025;  0.180,  0.973,  0.197; -0.010, -0.202,  0.995]
Compose the input transformation with average_inv.aff ...
Saving the output transformation as /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.aff
inFile = /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.nii.gz
outFile = /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk_aff.nii.gz
interpolation option is LEI
reorientOption = PPD
Reading /home/virk/devs/WorkProjects/PGI/PGIMRI/notebooks/dti_dtitk.ni

In [ ]:
# !TVglyphView -in mean_affine3.nii.gz -scale 0.001

In [29]:
!TVglyphView -in {dtitk_aff_path} -scale 0.001

Reading dti_dtitk_aff.nii.gz ... Done in 0.059151s
press the key 'H' to print a list of the available control keystrokes
total number of colors = 262144
processing the selected axial slice ... done rendering 


#### 5. Deformable alignment with template refinement

In [30]:
!TVtool -in mean_affine3.nii.gz -tr
!BinaryThresholdImageFilter mean_affine3_tr.nii.gz mask.nii.gz 0.01 100 1 0

Reading mean_affine3.nii.gz ... Done in 0.060658s
Computing the tensor trace (mean diffusion) map ... Done in 0.004092s
Writing mean_affine3_tr.nii.gz ... Done in 0.040789s


In [34]:
# Really important to check that the mask is appropriate before embarking on the next most time-consuming step.
# !dti_diffeomorphic_population mean_affine3.nii.gz subjects_list_aff.txt mask.nii.gz 0.002 


!dti_diffeomorphic_reg {template_path} {dtitk_aff_path} {template_mask_path} 1 5 0.002

registering dti_dtitk_aff.nii.gz to ixi_aging_template.nii.gz ...
starting at Sun 15 Aug 2021 03:53:14 PM IST

iteration 1 begins ...
Similarity Measure : DDS, Reorient Option : FS
Reading ixi_aging_template.nii.gz ... Done in 0.073992s
Reading dti_dtitk_aff_diffeo.nii.gz ... Done in 0.060887s
Reading ixi_aging_template_brain_mask.nii.gz ... Done in 0.001898s
starting level 2, ending level 5
Level 2
Piecewise Affine Setup: size = 8, 8, 4; vsize = 28, 28, 36
pDim = 3072; scalings : img = 0.06, prior = 0.1, reg = 0.04; ftol = 0.002
before : sum = 8918.52, img = 148642, prior = 0, reg = 0
iter 0: sum = 8918.52, img = 148642, prior = 0, reg = 0
iter 1: sum = 8522.01, img = 138225, prior = 30.0961, reg = 5637.05
iter 2: sum = 8477.13, img = 139805, prior = 36.9991, reg = 2128.68
iter 3: sum = 8477.04, img = 139708, prior = 36.7182, reg = 2272.45
iter = 27, iterGrad = 4
after : sum = 8477.03, img = 139684, prior = 36.6524, reg = 2308.31
Level 3
Piecewise Affine Setup: size = 16, 16, 8; vsize

iter 18: sum = 5267.96, img = 83689.5, prior = 1661.05, reg = 2697.26
iter = 139, iterGrad = 19
after : sum = 5263.43, img = 83679.4, prior = 1640.25, reg = 2640.43
Level 4
Piecewise Affine Setup: size = 32, 32, 16; vsize = 7, 7, 9
pDim = 196608; scalings : img = 0.06, prior = 0.09, reg = 0.036; ftol = 0.002
before : sum = 5263.43, img = 83679.9, prior = 1641.19, reg = 2636.95
iter 0: sum = 5263.43, img = 83679.9, prior = 1641.19, reg = 2636.95
iter 1: sum = 5139.77, img = 81642.5, prior = 1452.44, reg = 3069.35
iter 2: sum = 5106.01, img = 81491, prior = 1431.68, reg = 2436.18
iter 3: sum = 5070.48, img = 80078.8, prior = 1478.02, reg = 3686.78
iter 4: sum = 5052.26, img = 79145.1, prior = 1592.84, reg = 4449.94
iter 5: sum = 5040.77, img = 78782, prior = 1739.4, reg = 4369.52
iter 6: sum = 5033.11, img = 78611.5, prior = 1867.75, reg = 4120.03
iter = 57, iterGrad = 7
after : sum = 5031.32, img = 78615.2, prior = 1867.12, reg = 4066
Level 5
Piecewise Affine Setup: size = 64, 64, 32; v

Reading dti_dtitk_aff_diffeo.df.nii.gz ... Done in 0.453651s
Writing dti_dtitk_aff_to_ixi_aging_template.5.df.nii.gz ... Done in 3.38629s
inFile = dti_dtitk_aff_to_ixi_aging_template.5.df.nii.gz
Reading dti_dtitk_aff_to_ixi_aging_template.5.df.nii.gz ... Done in 0.464224s
outFile = dti_dtitk_aff_to_ixi_aging_template.5.df_jac.nii.gz
Writing dti_dtitk_aff_to_ixi_aging_template.5.df_jac.nii.gz ... Done in 2.27006s
JACOBIAN STATISTICS: after previous iteration mean = 1.00811 min = 0.228159 max = 3.72345 # of voxels = 1.78546e+06 after current iteration mean = 1.0092 min = 0.121706 max = 5.52223 # of voxels = 1.78546e+06
inFile = dti_dtitk_aff.nii.gz
outFile = dti_dtitk_aff_diffeo_current.nii.gz
transFile = dti_dtitk_aff_to_ixi_aging_template.5.df.nii.gz
df option is FD
interpolation option is LEI
reorientOption = PPD
Reading dti_dtitk_aff.nii.gz ... Done in 0.059701s
Reading dti_dtitk_aff_to_ixi_aging_template.5.df.nii.gz ... Done in 0.455672s
backward resampling ...time consumed = 1.6539

iter 20: sum = 3402.87, img = 53283.1, prior = 1608.97, reg = 4556.17
iter 21: sum = 3394.37, img = 53138.6, prior = 1661.7, reg = 4431.24
iter 22: sum = 3387.91, img = 53080.5, prior = 1675.19, reg = 4273.72
iter = 187, iterGrad = 23
after : sum = 3385.3, img = 53067.5, prior = 1655.89, reg = 4246.28
Level 5
Piecewise Affine Setup: size = 64, 64, 32; vsize = 3.5, 3.5, 4.5
pDim = 1572864; scalings : img = 0.06, prior = 0.06, reg = 0.024; ftol = 0.002
before : sum = 3385.3, img = 53066.9, prior = 1654.93, reg = 4249.65
iter 0: sum = 3385.3, img = 53066.9, prior = 1654.93, reg = 4249.65
iter 1: sum = 3275.25, img = 51209.6, prior = 1529.06, reg = 4622.02
iter 2: sum = 3242.28, img = 50999.7, prior = 1468.89, reg = 3923.59
iter 3: sum = 3187.36, img = 49300.3, prior = 1345.93, reg = 6191.2
iter 4: sum = 3157.14, img = 48198.6, prior = 1354.09, reg = 7665.85
iter 5: sum = 3145.54, img = 47913.9, prior = 1419.56, reg = 7730.39
iter 6: sum = 3133.03, img = 47681.5, prior = 1554.99, reg = 745

In [ ]:
!VolumeInfo mean_affine3.nii.gz

In [ ]:
!VolumeInfo {template_path}

In [45]:
!TVglyphView -in dti_dtitk_aff_diffeo.nii.gz

** ERROR (nifti_image_read): failed to find header file for 'dti_dtitk_aff_diffeo.nii.gz'


In [44]:
!TVglyphView -in /home/virk/Downloads/dtitk_samples/runtime/tensor_aff_diffeo.nii.gz

Reading /home/virk/Downloads/dtitk_samples/runtime/tensor_aff_diffeo.nii.gz ... Done in 0.077252s
press the key 'H' to print a list of the available control keystrokes
total number of colors = 262144
processing the selected axial slice ... done rendering 


In [46]:
!ls

 dti_dtitk.nii.gz
 dti_dtitk_nonSPD.nii.gz
 dti_dtitk_norm.nii.gz
 dti_dtitk_norm_non_outliers.nii.gz
 dti_dtitk_scaled.nii.gz
 dti_dtitk_scaled_nonSPD.nii.gz
 dti_dtitk_scaled_spd.aff
 dti_dtitk_scaled_spd_aff.nii.gz
 dti_dtitk_scaled_spd.nii.gz
 dti_dtitk_scaled_spd_norm.nii.gz
 DTI_ITC.ipynb
 dti_L1.nii.gz
 dti_L2.nii.gz
 dti_L3.nii.gz
'DTI-TK image registration_ALL_STEPS.ipynb'
'DTI-TK image registration example.ipynb'
'DTI-TK image registration.ipynb'
 dti_V1.nii.gz
 dti_V2.nii.gz
 dti_V3.nii.gz
 ixi_aging_template_brain_mask.nii.gz
 ixi_aging_template.nii.gz
 mean_initial.nii.gz
 reference_axial_tensor.png
 reference_coronal_tensor.png
 subjects_list_aff_diffeo.txt
 subjects_list_aff.txt
 subjects_list.txt
